## Objetivo 2: poder preditivo, regressão via random forest

A ideia dessa segunda parte é treinar um modelo mais robusto visando o poder preditivo e a obtenção de um modelo para uso em produção (uso real em uma aplicação web)

In [107]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"
import plotly.express as px

# Carregando os dados
df = pd.read_csv('../1. Dados/base.csv')

In [108]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split

from category_encoders import JamesSteinEncoder


- Transforme o sua variável de interesse usando uma transformação logarítmica (ex: y_log=np.log(y))

In [109]:
df = df.assign(log_PrecoVenda = lambda x: np.log(x.PrecoVenda))

In [110]:
X = df.drop(columns=['PrecoVenda','log_PrecoVenda'])
y = df['log_PrecoVenda']

In [111]:
sns.set(font_scale = 1.3) # Para aumentar o tamanho da fonte
coluna = "log_PrecoVenda" # Coluna a ser representada

fig = px.histogram(y)
fig.show()

Separação dos dados em treinamento e validação: os dados devem ser separados em treino, validação e teste, na fase de exploração e modelagem você pode avaliar o modelo usando o dataset de validação para evitar overfitting, e depois, com estudo fechado aplicar as métricas de avaliação no dataset de teste (simulando a performance em exemplos nunca vistos). Para esse caso você deve separar os datasets usando a função train_test_split do sklearn, usando como random state o número 42:

- Primeiro use a função função train_test_split  para separar 70% para treino e 30% para validação e teste

In [112]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.3 , random_state = 42)

- Segundo, aplique novamente essa função para quebrar esses 30% em dois datasets, sendo 50% para teste e 50% para validação. Assim obtendo 70% para treino, 15% para validação e 15% para teste

In [113]:
X_val , X_test , y_val , y_test = train_test_split(X_test , y_test , test_size = 0.5 , random_state = 42)

drop_feat --> Colunas de baixo impacto no modelo, estão sendo removidas

numeric_features --> Colunas numericas, seguirão por um tipo de tratamento no Pipeline

categorical_features --> Colunas Categoricas, seguirão por outro tipo de tratamento no Pipeline

In [114]:
drop_feat =  ['Aquecimento', 'ArCentral','AreaPiscina', 'AreaVaranda3Estacoes', 'BaixaQualiAreaAcab',
            'BanheiroPorao',  'Beco', 'BedroomAbvGr', 'CondicaoGaragem', 'CondicaoPorao',
            'Id','CondicaoExterna','KitchenAbvGr', 'CondicaoVenda', 'Estrada1','Funcionalidade',
            'InstalacaoEletrica', 'LavaboPorao', 'LocalGaragem', 'MaterialTelhado', 'Outros',
            'AreaVarandaFechada','AreaAcabPorao2','InclinacaoLote','ConfigLote','TipoAcabPorao1',
            'FormaProp','AnoVenda','AreaDeck','QualdiadeLareira','Bairro',
            'PID', 'PlanoProp', 'QualidadeCerca', 'QualidadeGaragem', 'QualidadePiscina', 'Rua',
            'Servicos', 'TipoAlvenaria', 'TipoHabitacao', 'TipoVenda','TipoTelhado', 'ValorOutros'
            ]

In [115]:
numeric_features = ['Fachada','TamanhoLote','Qualidade','Condicao','AnoConstrucao','AnoReforma',
                    'AreaAlvenaria','AreaAcabPorao1','AreaAcabPorao2','AreaInacabPorao','AreaPorao',
                    'AreaTerreo','Area2Andar','BaixaQualiAreaAcab','AreaConstruida','BanheiroPorao',
                    'LavaboPorao','Banheiro','Lavabo','BedroomAbvGr','KitchenAbvGr','TotalQuartos',
                    'Lareira','AnoGaragem','CarrosGaragem','AreaGaragem','AreaDeck','AreaVarandaAberta',
                    'AreaVarandaFechada','AreaVaranda3Estacoes','AreaAlpendre','AreaPiscina',
                    'ValorOutros','MesVenda','AnoVenda']

In [116]:
categorical_features = ['ClasseImovel','ClasseZona','Rua','Beco','FormaProp','PlanoProp',
                        'Servicos','ConfigLote','InclinacaoLote','Estrada1','Estrada2','TipoHabitacao',
                        'EstiloHabitacao','TipoTelhado','MaterialTelhado','Exterior1','Exterior2',
                        'TipoAlvenaria','QualidadeCobertura','CondicaoExterna','TipoFundacao','AlturaPorao',
                        'CondicaoPorao','ParedePorao','TipoAcabPorao1','TipoAcabPorao2','Aquecimento',
                        'QualidadeAquecimento','ArCentral','InstalacaoEletrica','QualidadeCozinha',
                        'Funcionalidade','QualdiadeLareira','LocalGaragem','AcabamentoGaragem',
                        'QualidadeGaragem','CondicaoGaragem','EntradaPavimentada','QualidadePiscina',
                        'QualidadeCerca','Outros','TipoVenda','CondicaoVenda']

In [117]:
for i in drop_feat:
    try:        
        categorical_features.remove(i)
    except ValueError:
        pass
    try:
        numeric_features.remove(i)       
    except ValueError:
        pass

In [118]:
from sklearn import set_config
set_config(display='diagram')

- Aplique as transformações nas variáveis categóricas que você julgar necessárias (One hot encoding, ordinal encoding e etc…), as transformações também devem ser fitadas usando o dataset de treino para depois serem aplicadas nos datasets de validação e teste, isso evitará data leakage (Dica: usar os transformers do sklearn) 

- Impute os valores faltantes das variáveis numéricas com a mediana e os valores faltantes das variáveis categóricas com a moda, os imputers devem ser fitados usando o dataset de treino para depois serem aplicadas nos datasets de validação e teste, isso evitará data leakage (Dica: usar os simpleimputer do sklearn) 

In [119]:
numeric_transformer = Pipeline(steps=[
                                     ('imputar_mediana', SimpleImputer(strategy='median')),
                                     ('stdscaler', PowerTransformer())
                                     ])

categorical_transformer = Pipeline(steps=[                                         
                                         ('onehotenc', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value= -1)),
                                         ('imputar_mais_frequente', SimpleImputer(strategy='most_frequent')),
                                         ('stdscaler', StandardScaler())
                                         ])

In [120]:
col_transformer = ColumnTransformer(transformers=[('numeric_processing',numeric_transformer, numeric_features),
                                                  ('categorical_processing', categorical_transformer, categorical_features)
                                                  ], remainder='drop')

In [121]:
pipeline = Pipeline([
                     ('transform_column', col_transformer),
                    ])

In [122]:
pipeline.fit(X_train)

Pipeline(steps=[('transform_column',
                 ColumnTransformer(transformers=[('numeric_processing',
                                                  Pipeline(steps=[('imputar_mediana',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('stdscaler',
                                                                   PowerTransformer())]),
                                                  ['Fachada', 'TamanhoLote',
                                                   'Qualidade', 'Condicao',
                                                   'AnoConstrucao',
                                                   'AnoReforma',
                                                   'AreaAlvenaria',
                                                   'AreaAcabPorao1',
                                                   'AreaInacabPorao',
                                                   'AreaPorao', 'AreaTerreo',
                                                   'Ar...
                                                                  ('imputar_mais_frequente',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['ClasseImovel', 'ClasseZona',
                                                   'Estrada2',
                                                   'EstiloHabitacao',
                                                   'Exterior1', 'Exterior2',
                                                   'QualidadeCobertura',
                                                   'TipoFundacao',
                                                   'AlturaPorao', 'ParedePorao',
                                                   'TipoAcabPorao2',
                                                   'QualidadeAquecimento',
                                                   'QualidadeCozinha',
                                                   'AcabamentoGaragem',
                                                   'EntradaPavimentada'])]))])

In [123]:
transformed_train=pipeline.transform(X_train)

In [124]:
transformed_train.shape

(1022, 38)

In [125]:
for name, estimator, features in pipeline.named_steps['transform_column'].transformers_:
  print(name)
  print(features)

numeric_processing
['Fachada', 'TamanhoLote', 'Qualidade', 'Condicao', 'AnoConstrucao', 'AnoReforma', 'AreaAlvenaria', 'AreaAcabPorao1', 'AreaInacabPorao', 'AreaPorao', 'AreaTerreo', 'Area2Andar', 'AreaConstruida', 'Banheiro', 'Lavabo', 'TotalQuartos', 'Lareira', 'AnoGaragem', 'CarrosGaragem', 'AreaGaragem', 'AreaVarandaAberta', 'AreaAlpendre', 'MesVenda']
categorical_processing
['ClasseImovel', 'ClasseZona', 'Estrada2', 'EstiloHabitacao', 'Exterior1', 'Exterior2', 'QualidadeCobertura', 'TipoFundacao', 'AlturaPorao', 'ParedePorao', 'TipoAcabPorao2', 'QualidadeAquecimento', 'QualidadeCozinha', 'AcabamentoGaragem', 'EntradaPavimentada']
remainder
[0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 21, 22, 25, 28, 31, 33, 36, 39, 41, 42, 45, 47, 48, 51, 52, 55, 57, 58, 63, 64, 66, 68, 69, 71, 72, 73, 74, 75, 77, 78, 79]


In [126]:
pipeline = Pipeline([
                     ('transform_column', col_transformer),
                     ('RForest', RandomForestRegressor(n_estimators = 150,max_features=0.3, n_jobs=-1))
                    ])

In [127]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transform_column',
                 ColumnTransformer(transformers=[('numeric_processing',
                                                  Pipeline(steps=[('imputar_mediana',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('stdscaler',
                                                                   PowerTransformer())]),
                                                  ['Fachada', 'TamanhoLote',
                                                   'Qualidade', 'Condicao',
                                                   'AnoConstrucao',
                                                   'AnoReforma',
                                                   'AreaAlvenaria',
                                                   'AreaAcabPorao1',
                                                   'AreaInacabPorao',
                                                   'AreaPorao', 'AreaTerreo',
                                                   'Ar...
                                                                   StandardScaler())]),
                                                  ['ClasseImovel', 'ClasseZona',
                                                   'Estrada2',
                                                   'EstiloHabitacao',
                                                   'Exterior1', 'Exterior2',
                                                   'QualidadeCobertura',
                                                   'TipoFundacao',
                                                   'AlturaPorao', 'ParedePorao',
                                                   'TipoAcabPorao2',
                                                   'QualidadeAquecimento',
                                                   'QualidadeCozinha',
                                                   'AcabamentoGaragem',
                                                   'EntradaPavimentada'])])),
                ('RForest',
                 RandomForestRegressor(max_features=0.3, n_estimators=150,
                                       n_jobs=-1))])

In [128]:
print("pipeline train r2_score: %0.3f" % pipeline.score(X_train, y_train))

pipeline train r2_score: 0.981


In [129]:
y_pred = pipeline.predict(X_val)

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("r2_score: %0.3f" % r2_score(np.exp(y_val), np.exp(y_pred)))
print("MAE: %0.3f" % mean_absolute_error(np.exp(y_val), np.exp(y_pred)))
print("RMSE: %0.3f" % mean_squared_error(np.exp(y_val), np.exp(y_pred), squared=False))

r2_score: 0.905
MAE: 16079.568
RMSE: 24484.476


- Compute a importância das features no dataset de validação (usar permutation_importance do sklearn), usando a importância das features remova do seu treinamento as features menos importantes para que o seu modelo tenha no máximo 40 features, verifique novamente a performance com esse número reduzido de features (isso pode melhorar a performance e a velocidade do seu modelo)

In [24]:
from sklearn.inspection import permutation_importance

scoring = ['neg_mean_absolute_error','neg_root_mean_squared_error']
r_multi = permutation_importance(
    pipeline, X_val, np.exp(y_val), n_repeats=50, random_state=42, scoring=scoring, n_jobs=-1)

for metric in r_multi:
    print(" ")
    print(f"{metric}")
    print(" ")
    r = r_multi[metric]
    for i in r.importances_mean.argsort()[::-1]:
        if abs(r.importances_mean[i]) >= 0.0025:
            print(f"    {X_val.columns[i]:<8}:  "
                    f"{r.importances_mean[i]:.3f}"
                    f" +/- {r.importances_std[i]:.3f}")

 
neg_mean_absolute_error
 
    Qualidade:  0.008 +/- 0.002
    AreaConstruida:  0.008 +/- 0.001
    AlturaPorao:  0.006 +/- 0.001
    CarrosGaragem:  0.004 +/- 0.001
    AreaGaragem:  0.004 +/- 0.001
    QualidadeCozinha:  0.003 +/- 0.000
    AnoGaragem:  0.003 +/- 0.000
    AreaAcabPorao1:  0.003 +/- 0.001
    AreaTerreo:  0.003 +/- 0.001
 
neg_root_mean_squared_error
 
    Qualidade:  0.042 +/- 0.004
    AreaConstruida:  0.031 +/- 0.003
    CarrosGaragem:  0.011 +/- 0.001
    AreaGaragem:  0.010 +/- 0.001
    AlturaPorao:  0.010 +/- 0.001
    AreaTerreo:  0.009 +/- 0.001
    AnoConstrucao:  0.009 +/- 0.001
    QualidadeCobertura:  0.007 +/- 0.001
    AreaPorao:  0.007 +/- 0.001
    TamanhoLote:  0.007 +/- 0.001
    AreaAcabPorao1:  0.007 +/- 0.001
    AnoGaragem:  0.005 +/- 0.001
    QualidadeCozinha:  0.005 +/- 0.001
    Banheiro:  0.003 +/- 0.001
    AnoReforma:  0.003 +/- 0.000


- Adicione um breve texto com sua interpretação em relação à importância das features

# Finalmente, compute as métricas de avaliação no dataset de teste para obter o proxy de performance do seu modelo em um ambiente em produção (ambiente real online).

Avaliação do modelo de regressão: Para fazer a avaliação do seu modelo você deve aplicar métricas de avaliação no dataset de validação (e no final do estudo no dataset de teste), as seguintes métricas são comuns em modelos de regressão:

- R²: pense nesse score como uma medida do desempenho do nosso modelo em comparação com um modelo trivial que retorna sempre  a média para qualquer previsão solicitada. (o valor de 1.0 representa um modelo perfeito, já um valor de 0.0 representa um modelo equivalente a um modelo aleatório)

- Valor absoluto médio (MAE): que é apenas a diferença absoluta média entre os valores previstos e verdadeiros. O valor absoluto evita que desvios negativos e positivos se cancelem. 

- Em vez de tomar o valor absoluto, poderíamos elevar ao quadrado as diferenças, dando-nos o erro quadrático médio (MSE). Elevar a diferença também tem o efeito de enfatizar quaisquer previsões que estejam muito longe de seus verdadeiros valores.

- Para ignorar algumas previsões significativamente desviantes (outliers), é melhor usar o MAE no lugar MSE. Tudo depende do que você está buscando

- Como as unidades do MSE são o quadrado das unidades da variável de interesse, é útil usar a raiz do erro quadrático médio (RMSE) como métrica de avaliação

In [82]:
y_pred = pipeline.predict(X_test)

In [83]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("r2_score: %0.3f" % r2_score(np.exp(y_test), np.exp(y_pred)))
print("MAE: %0.3f" % mean_absolute_error(np.exp(y_test), np.exp(y_pred)))
print("RMSE: %0.3f" % mean_squared_error(np.exp(y_test), np.exp(y_pred), squared=False))

r2_score: 0.881
MAE: 16906.086
RMSE: 30040.059


# Testando diferentes hiperparâmetros do seu Random Forest: Primeiro, aumente o número de árvores (n_estimators) até que a precisão pare de melhorar. Em seguida, usando o número de árvores da primeira etapa, tente alguns valores de max_features e escolha aquele que forneça a melhor métrica. Finalmente, usando o melhor max_features, execute min_samples_leaf de 1 até 15, novamente escolhendo o melhor.

- n_estimators (10, 30, 50, 70, 100, 150, 200)

- max_features ('sqrt', 0.1 até 0.6)

- min_samples_leaf(1 até 15)

In [113]:
grid_param = [
{"RForest": [RandomForestRegressor()],
"RForest__n_estimators":[10, 30, 50, 70, 100, 150, 200],
"RForest__max_features": ['sqrt', 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],  
"RForest__min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}]
gridsearch = GridSearchCV(pipeline, grid_param, n_jobs=-1) 

In [114]:
best_model = gridsearch.fit(X_train,y_train)

In [115]:
print("best_model train r2_score: %0.3f" % best_model.score(X_train, y_train))

best_model train r2_score: 0.963


In [116]:
y_pred = pipeline.predict(X_val)

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("r2_score: %0.3f" % r2_score(np.exp(y_val), np.exp(y_pred)))
print("MAE: %0.3f" % mean_absolute_error(np.exp(y_val), np.exp(y_pred)))
print("RMSE: %0.3f" % mean_squared_error(np.exp(y_val), np.exp(y_pred), squared=False))

r2_score: 0.909
MAE: 15668.263
RMSE: 23860.556


In [96]:
gridsearch.best_estimator_

Pipeline(steps=[('transform_column',
                 ColumnTransformer(transformers=[('drop_columns', 'drop',
                                                  ['Aquecimento', 'ArCentral',
                                                   'AreaPiscina',
                                                   'AreaVaranda3Estacoes',
                                                   'BaixaQualiAreaAcab',
                                                   'BanheiroPorao', 'Beco',
                                                   'BedroomAbvGr',
                                                   'CondicaoGaragem',
                                                   'CondicaoPorao', 'Id',
                                                   'CondicaoExterna',
                                                   'KitchenAbvGr',
                                                   'CondicaoVenda', 'Estrada1',
                                                   'Funcionalidade',
                                                   'InstalacaoEletrica'...
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['ClasseImovel', 'ClasseZona',
                                                   'Estrada2',
                                                   'EstiloHabitacao',
                                                   'Exterior1', 'Exterior2',
                                                   'QualidadeCobertura',
                                                   'TipoFundacao',
                                                   'AlturaPorao', 'ParedePorao',
                                                   'TipoAcabPorao2',
                                                   'QualidadeAquecimento',
                                                   'QualidadeCozinha',
                                                   'AcabamentoGaragem',
                                                   'EntradaPavimentada'])])),
                ('RForest',
                 RandomForestRegressor(max_features=0.3, n_estimators=150))])

# Adicione uma conclusão para fechar o seu case